# Assignment 3 - GEE API and folium mapping

## Aim: 
In our group project we want to create a python package for generating vegetation indices, mapping, time-series (something like this). Because I haven't worked with the python API for GEE yet, so I want to use this notebook to filter data from the Sentinel-2 collection and choose the latest image with low cloud percentage. Next step is to calculate the NDVI and display it using either the folium package or the geemap functions (which are build upon folium themselves). 

## Installation of the GEE-API in new environment 
- Workflow based on: https://developers.google.com/earth-engine/guides/python_install-conda 
- Created a new conda environment named: 'gee_test' that has all needed packages installed -> see exported environment file.


## GEE Initialization

In [26]:
# Import needed libraries 
import ee
import folium

In [27]:
# In order to use GEE (its EO data collections AND computing resources) we have to authentificate
# and to initialize the project we want to work in. 
ee.Authenticate()
ee.Initialize(project='ee-davidhansen896')

In [28]:
# Check initialization of GEE 
print(ee.String('Hello From the Earth Engine servers!').getInfo())

Hello From the Earth Engine servers!


## Create AOI 

In [29]:
# Create a rectangle from latlong-coordinates: Golf of Morbihan - Vannes 
aoi = ee.Geometry.Rectangle([
    -2.9, # lon min
    47.48, # lat min
    -2.6, # lon max
    47.63 # lat max
]) 
aoi.getInfo()

{'type': 'Polygon',
 'coordinates': [[[-2.9, 47.48],
   [-2.6, 47.48],
   [-2.6, 47.63],
   [-2.9, 47.63],
   [-2.9, 47.48]]]}

## Subsetting the Sentinel-2 harmonized surface reflectance collection 

In [30]:
# Set start and end dates for our image selection
start_date = '2023-06-01' 
end_date = '2023-08-31'

# Select S2 selection
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

# Filter selection based on aoi boundaries, dates and cloud percentage <= 10 (ee.Filter.lte()) 
collection = s2.filterBounds(aoi).filterDate(start_date, end_date).filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 10))

# Print the number of images in the collection
print(collection.size().getInfo())

7


## Reduce collection and NDVI calculation
We decide to first reduce the collection to a single image by calculating the mean for every pixel based on the corresponding pixels in all - in this case - 7 images. After that we select the red and infrared band of the reduced image and calcualte the NDVI. 

In [31]:
# reduce the image using mean 
mean_img = collection.mean()

In [32]:
# Extract bands
red = mean_img.select('B4')
nir = mean_img.select('B8') 

# Calculate NDVI and save as 'NDVI' in an image object
ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
#print(ndvi.getInfo())

nvdi = ndvi.clip(aoi)

## Mapping 
First we use the following function for adding ee_layers to a folium map: 
Using 'getMapId()' the MapId dictionary with given visual parameters is returned. We use this to fetch the tiles url_format. Which then is used to add the images to the map. 

In [33]:
# Create folium mapping function for GEE images: From GEE Documentary
# https://developers.google.com/earth-engine/guides/python_install-colab#interactive_map

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = "Map Data © Google Earth Engine",
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [34]:
# Get the center of the map by refering to the centroid information of the aoi rectangle
map_center = [aoi.centroid().getInfo()['coordinates'][1], aoi.centroid().getInfo()['coordinates'][0]] # latitude comes first 

In [35]:
# Create map output of NDVI 
ndvi_map = folium.Map(location=map_center, zoom_level=5) # Zoom level to focus intial point of the map to aoi center

# Set visual parameters NDVI
vis_params_ndvi = {
    'bands': ['NDVI'], 
    'min': 0, 
    'max': 1, 
    'palette': ['#2A20E5', '#2440CB', '#1E60B1', '#188097',
                '#12A07D', '#0FB070', '#0CC063', '#06E049',
               '#03F03C', '#00FF2F']
}

# Add NDVI to map
ndvi_map.add_ee_layer(ndvi, vis_params_ndvi, 'NDVI')

# Add a layer control panel to the map.
ndvi_map.add_child(folium.LayerControl())

# Display map 
display(ndvi_map)